In [14]:
import os
import shutil

def copy_markdown_files(src_dir, dest_dir):
    """递归地将所有子文件夹中的 Markdown 文件复制到目标目录"""
    if not os.path.exists(dest_dir):
        os.makedirs(dest_dir)  # 如果目标目录不存在，则创建

    # 遍历源目录及其子文件夹
    for root, dirs, files in os.walk(src_dir):
        for file in files:
            if file.endswith(".md"):  # 只处理 .md 文件
                # 构造源文件路径
                src_file = os.path.join(root, file)
                # 构造目标文件路径
                dest_file = os.path.join(dest_dir, file)
                # 直接复制文件（如果目标文件存在，则会覆盖）
                shutil.copy(src_file, dest_file)
                print(f"复制文件: {src_file} 到 {dest_file}")

# 使用示例
src_directory = "outputs2/"  # 源目录路径
dest_directory = "all_md/"  # 目标目录路径

#copy_markdown_files(src_directory, dest_directory)


In [15]:
import os
import json
from langchain_text_splitters import MarkdownHeaderTextSplitter
from multiprocessing import Pool, Manager
from tqdm import tqdm  # 进度条库

headers_to_split_on = [
    ("#", "Header")
]

def process_markdown_file(file_path):
    """处理单个 Markdown 文件，返回 result1 和 result2"""
    try:
        with open(file_path, "r", encoding="utf-8") as file:
            markdown_content = file.read()

        markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on)
        chunks = markdown_splitter.split_text(markdown_content)
        
        header_key = [chunks[i].metadata['Header'].lower() for i in range(1,len(chunks))]
        index_1 = header_key.index('introduction')+1 if 'introduction' in header_key else None
        index_1 = index_1 if index_1 else header_key.index('background')+1 if 'background' in header_key else None
        index_2 = header_key.index('discussion') if 'discussion' in header_key else None
        title = chunks[0].metadata['Header'] if chunks[0].metadata else chunks[1].metadata['Header'] 

        results1 = {
            "text": chunks[index_1].page_content if index_1 is not None else ' ',
            "meta": {"title": title, "name": file_path.split('/')[1], "type": "introduction"}
        }
        results2 = {
            "text": chunks[index_2].page_content if index_2 is not None else ' ',
            "meta": {"title": title, "name": file_path.split('/')[1], "type": "discussion"}
        }
        
        return results1, results2
    except Exception as e:
        print(f"Error processing file {file_path}: {e}")
        return None, None

def write_to_jsonl(results, output_file):
    """将结果写入 JSONL 文件"""
    with open(output_file, "a", encoding="utf-8") as file:
        for result in results:
            json.dump(result, file, ensure_ascii=False)
            file.write("\n")

def worker(args):
    """工作进程函数，将结果返回"""
    file_path, result_list1, result_list2 = args
    result1, result2 = process_markdown_file(file_path)
    if result1 and result2:
        result_list1.append(result1)
        result_list2.append(result2)

def process_files_in_parallel(input_dir, output_file1, output_file2, num_workers=4):
    """并行处理文件夹中的 Markdown 文件，分别写入两个 JSONL 文件"""
    # 获取所有 Markdown 文件路径
    markdown_files = [os.path.join(input_dir, f) for f in os.listdir(input_dir) if f.endswith(".md")]

    # 使用 Manager 创建共享列表
    with Manager() as manager:
        result_list1 = manager.list()
        result_list2 = manager.list()
        
        # 创建进程池
        with Pool(processes=num_workers) as pool:
            # 使用 tqdm 包裹任务以显示进度条
            with tqdm(total=len(markdown_files), desc="Processing files") as pbar:
                for _ in pool.imap_unordered(worker, [(f, result_list1, result_list2) for f in markdown_files]):
                    pbar.update()
        
        # 将结果写入 JSONL 文件
        write_to_jsonl(result_list1, output_file1)
        write_to_jsonl(result_list2, output_file2)

In [ ]:
if __name__ == "__main__":
    input_directory = 'all_md/'
    output_jsonl1 = 'introduction.jsonl'
    output_jsonl2 = 'discussion.jsonl'
    
    process_files_in_parallel(input_directory, output_jsonl1, output_jsonl2, num_workers=64)

Processing files:   6%|█▏                 | 1877/30242 [00:08<02:03, 229.05it/s]

Error processing file all_md/oskarsson2018.md: list index out of range


Processing files:  15%|██▊                | 4423/30242 [00:19<01:54, 224.77it/s]

Error processing file all_md/jackson2017.md: list index out of range


Processing files:  42%|███████▍          | 12568/30242 [00:51<01:01, 289.44it/s]

Error processing file all_md/decastro2018.md: list index out of range


Processing files:  58%|██████████▍       | 17638/30242 [01:13<00:56, 224.94it/s]

Error processing file all_md/bailey2014.md: list index out of range


Processing files:  62%|███████████▏      | 18761/30242 [01:18<00:49, 229.75it/s]